### Membaca data

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)

import time

In [2]:
# Importing data
data = pd.read_csv('clean_data.csv')         # data primer    
del data['Unnamed: 0']
data

,1,10,100,101,102,103,104,105,106,107,...,91,92,93,94,95,96,97,98,99,radiant_win
0,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
511371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
511372,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
511373,0,0,0,0,0,0,-1,0,0,0,...,0,0,0,0,0,-1,0,0,0,0


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split dataset menjadi train, validation, dan test
# from sklearn.model_selection import train_test_split
# train_inputs, X_test, train_targets, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
# val_inputs, test_inputs, val_targets, test_targets = train_test_split(X_test, y_test, test_size = 0.40, random_state = 0)

### Kfold

In [4]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

10

### Model

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm

#### Mendefinisikan model

In [6]:
# Set the output and hidden layer sizes
input_neuron= 121
hidden_neuron = 300
output_neuron = 1


init_mode = 'random_uniform'
dropout_rate = 0.4
weight_constraint = 3
    
# define how the model will look like
model = Sequential()
model.add(InputLayer(input_shape=(input_neuron,)))                                      # input layer
model.add(Dense(hidden_neuron, kernel_initializer=init_mode, activation='gelu',         # 1st hidden layer
                kernel_constraint=MaxNorm(weight_constraint)))
model.add(Dense(output_neuron, kernel_initializer=init_mode, activation='sigmoid'))     #  output layer

learning_rate = 0.001
momentum = 0.9

# Choose the optimizer and the loss function
optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 80
max_epochs = 10


#### Training & test model

In [7]:
kfold_accuracy = []
k = 1

for train_index, test_index in skf.split(X, y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print("K-%s -->" % (k))
    k+=1
    
    # fit the model
    model.fit(X_train, # train inputs
              y_train, # train targets
              batch_size=batch_size, # batch size
              epochs=max_epochs, 
              verbose = 2 
              )  
    
    print()
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print("\n")
    
    kfold_accuracy.append(test_accuracy)

K-1 -->
Epoch 1/10
5753/5753 - 10s - loss: 0.6725 - accuracy: 0.5809
Epoch 2/10
5753/5753 - 8s - loss: 0.6497 - accuracy: 0.6205
Epoch 3/10
5753/5753 - 8s - loss: 0.6413 - accuracy: 0.6311
Epoch 4/10
5753/5753 - 8s - loss: 0.6397 - accuracy: 0.6326
Epoch 5/10
5753/5753 - 8s - loss: 0.6393 - accuracy: 0.6330
Epoch 6/10
5753/5753 - 8s - loss: 0.6391 - accuracy: 0.6334
Epoch 7/10
5753/5753 - 8s - loss: 0.6390 - accuracy: 0.6333
Epoch 8/10
5753/5753 - 7s - loss: 0.6390 - accuracy: 0.6336
Epoch 9/10
5753/5753 - 7s - loss: 0.6389 - accuracy: 0.6334
Epoch 10/10
5753/5753 - 7s - loss: 0.6389 - accuracy: 0.6337

1599/1599 [==============================] - 1s 758us/step - loss: 0.6356 - accuracy: 0.6371


K-2 -->
Epoch 1/10
5753/5753 - 7s - loss: 0.6384 - accuracy: 0.6342
Epoch 2/10
5753/5753 - 7s - loss: 0.6384 - accuracy: 0.6343
Epoch 3/10
5753/5753 - 7s - loss: 0.6384 - accuracy: 0.6345
Epoch 4/10
5753/5753 - 8s - loss: 0.6384 - accuracy: 0.6342
Epoch 5/10
5753/5753 - 9s - loss: 0.6384 - acc

In [8]:
for i in range(len(kfold_accuracy)):
    print("Accuracy in k-%s : %s" % (i+1, kfold_accuracy[i]))

    
avg_acc = sum(kfold_accuracy) / len(kfold_accuracy)
print("\nAverage accuracy with kfold : %s" % (avg_acc))

Accuracy in k-1 : 0.6371387243270874
Accuracy in k-2 : 0.6321913003921509
Accuracy in k-3 : 0.6346747875213623
Accuracy in k-4 : 0.6334428191184998
Accuracy in k-5 : 0.6332864165306091
Accuracy in k-6 : 0.6337485313415527
Accuracy in k-7 : 0.6371707320213318
Accuracy in k-8 : 0.6348827481269836
Accuracy in k-9 : 0.6328685879707336
Accuracy in k-10 : 0.6362711787223816

Average accuracy with kfold : 0.6345675826072693
